In [1]:
import ipywidgets as widgets
from solve import solve, read_file
from io import StringIO


# maximal number of ingredients
n = 15

In [2]:
class Ingredient():
    def __init__(self, default_name):
        self.is_available = widgets.Checkbox(
            value=False,
            description='',
            disabled=False,
            layout=widgets.Layout(width='auto')
        )
        self.name = widgets.Text(
            value=f'{default_name}:',
            placeholder='Ingredient name',
            description='',
            disabled=True,
            layout=widgets.Layout(width='15rem')
        )
        self.is_known = widgets.Checkbox(
            value=False,
            description='known',
            disabled=True,
            layout=widgets.Layout(width='auto')
        )
        self.value = widgets.BoundedFloatText(
            value=0.0,
            min=0.0,
            max=100.0,
            step=0.1,
            description='',
            disabled=True,
            layout=widgets.Layout(width='6rem')
        )
        self.result = widgets.Label(
            value='',
            layout=widgets.Layout(width='20rem')
        )
        def refresh_ui(change):
            self.name.disabled = not self.is_available.value
            self.is_known.disabled = not self.is_available.value
            self.name.disabled = not self.is_available.value
            self.value.disabled = not self.is_available.value or not self.is_known.value
        self.is_available.observe(refresh_ui, names='value')
        self.is_known.observe(refresh_ui, names='value')
        hbox = widgets.HBox([self.is_available, self.name, self.is_known, self.value, widgets.Label(value="%"), self.result])
        display(hbox)


ingredients = [Ingredient(f"Ingredient {i+1}") for i in range(n)]

load_button = widgets.FileUpload(
    description='Load from file',
    tooltip='Load from file',
    disabled=False,
    button_style='', # change to 'success' or 'warning' after clicking
    multiple=False
)

solve_button = widgets.Button(
    description='Solve',
    tooltip='Solve',
    disabled=False,
    button_style='', # change to 'success' or 'warning' after clicking
    icon='check'
)

message = widgets.HTML(value="")
display(load_button, solve_button, message)


def do_load(change):
    # we don't have a file, so we use StringIO
    file_contents = load_button.data[0].decode('utf-8')
    f = StringIO(file_contents)
    try:
        names, amounts = read_file(f)
    except ValueError as e:
        message.value = str(e)
        load_button.button_style = 'danger'
        return
    if len(names) > n:
        message.value = f"Error: This notebook supports up to {n} ingredients, but the selected file has {len(names)}"
        load_button.button_style = 'danger'
        return
    for i in range(n):
        ingredients[i].is_available.value = False
        ingredients[i].name.value = ''
        ingredients[i].is_known.value = False
        ingredients[i].value.value = 0
        ingredients[i].result.value = ''
        if i < len(names):
            ingredients[i].is_available.value = True
            ingredients[i].name.value = names[i]
            if amounts[i] is not None:
                ingredients[i].is_known.value = True
                percentage = round(amounts[i] * 1000) / 10.
                ingredients[i].value.value = percentage
    load_button.button_style = 'success'
    solve_button.button_style = ''
    message.value = f"Loaded file"


load_button.observe(do_load, names='value')


def do_solve(b):
    amounts = [ingredients[i].value.value/100. if ingredients[i].is_known.value else None
               for i in range(n) if ingredients[i].is_available.value]
    try:
        res = solve(amounts)
    except ValueError as e:
        solve_button.button_style = 'danger'
        message.value = f"""
Error solving: {e}<br>
Make sure that the amounts are correct and sorted from largest to smallest.
"""
        return
    solve_button.button_style = 'success'
    for i in range(n):
        if not ingredients[i].is_available.value:
            continue
        low, high = res[i]
        if low < high:
            ingredients[i].result.value = f"{low: >7.2%} -{high: >7.2%}"
        else:
            ingredients[i].result.value = f"     {low: >7.2%}"


solve_button.on_click(do_solve)


FileUpload(value={}, description='Load from file')

Button(description='Solve', icon='check', style=ButtonStyle(), tooltip='Solve')

HTML(value='')